In [ ]:
# hide
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp training

<IPython.core.display.Javascript object>

In [ ]:
# export
from torch import nn
from fastai2.basics import (
    Recorder,
    CrossEntropyLossFlat,
    flatten_check,
    accuracy,
    Callback,
    DataBunch,
    Learner,
)
from fastai2.callback.schedule import *
from fastai2.callback.progress import *
from fastai2.callback.fp16 import *

<IPython.core.display.Javascript object>

# Training

> Training the model

In [ ]:
# export
class DualCrossEntropyLossFlat(CrossEntropyLossFlat):
    def __call__(self, inp, targ, **kwargs):
        pitches_targ, durations_targ = targ.transpose(0, 1)
        pitches_targ = pitches_targ.transpose(0, 1)[-1]
        durations_targ = durations_targ.transpose(0, 1)[-1]
        pitches_input = inp[0].transpose(0, 1)[-1]
        durations_input = inp[1].transpose(0, 1)[-1]
        pitches_loss = super().__call__(pitches_input, pitches_targ, **kwargs)
        durations_loss = super().__call__(durations_input, durations_targ, **kwargs)
        return (pitches_loss + durations_loss) / 2


def pitch_accuracy(inp, targ):
    targ_pitches, targ_durations = targ.transpose(0, 1)
    return accuracy(inp[0], targ_pitches)


def rhythm_accuracy(inp, targ):
    targ_pitches, targ_durations = targ.transpose(0, 1)
    return accuracy(inp[1], targ_durations)


def avg_accuracy(inp, targ):
    return (pitch_accuracy(inp, targ) + rhythm_accuracy(inp, targ)) / 2


class ResetTrainer(Callback):
    def begin_train(self):
        self.model.reset()

    def begin_validate(self):
        self.model.reset()

<IPython.core.display.Javascript object>

In [ ]:
# export


def get_learner(db: DataBunch, model: nn.Module) -> Learner:
    """
    Constructs a learner from a databunch, using DualCrossEntroyLossFlat as a loss function.
    """
    return Learner(
        db,
        model,
        loss_func=DualCrossEntropyLossFlat(),
        cb_funcs=ResetTrainer,
        metrics=[pitch_accuracy, rhythm_accuracy, avg_accuracy],
    )

<IPython.core.display.Javascript object>